In [ ]:
import numpy as np
import os
import random
import scipy
from scipy import signal
from numpy.fft import fft
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.initializers import he_normal
from tensorflow.python.keras.utils.vis_utils import plot_model

from sklearn.model_selection import train_test_split

import pandas as pd
import pickle

import seaborn as sns
sns.set(font='Yu Gothic')
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
def load_npy(hold_position):
    file_path = "../Output/validation/validation_" + hold_position + "/validation_" + hold_position
    xy_mean = np.load(file_path + "_glo_laccel_xy_mean.npy")
    xy_var = np.load(file_path + "_glo_laccel_xy_var.npy")
    z_mean = np.load(file_path + "_glo_laccel_z_mean.npy")
    z_var = np.load(file_path + "_glo_laccel_z_var.npy")
    z_skew = np.load(file_path + "_glo_laccel_z_skew.npy")
    z_kurtosis = np.load(file_path + "_glo_laccel_z_kurtosis.npy")
    z_lacc_FFT = np.load(file_path + "_glo_laccel_z_amplitude_frequency_range5Hz.npy")[:, 0:-1:2]
    z_gyro_FFT = np.load(file_path + "_glo_gyro_z_amplitude_frequency_range5Hz.npy")[:, 0:-1:2]
    result = np.concatenate([xy_mean.reshape([-1, 1]), xy_var.reshape([-1, 1]), z_mean.reshape([-1, 1]), z_var.reshape([-1, 1]), z_skew.reshape([-1, 1]), z_kurtosis.reshape([-1, 1]), z_lacc_FFT, z_gyro_FFT], axis=1)
    del xy_mean, xy_var, z_mean, z_var, z_skew, z_kurtosis, z_gyro_FFT
    return result

In [ ]:
def load_npy(hold_position):
    file_path = "../Output/validation/validation_" + hold_position + "/validation_" + hold_position
    xy_mean = np.load(file_path + "_laccel_xy_mean.npy")
    xy_var = np.load(file_path + "_laccel_xy_var.npy")
    z_mean = np.load(file_path + "_laccel_z_mean.npy")
    z_var = np.load(file_path + "_laccel_z_var.npy")
    z_skew = np.load(file_path + "_laccel_z_skew.npy")
    z_kurtosis = np.load(file_path + "_laccel_z_kurtosis.npy")
    z_lacc_FFT_sum = np.load(file_path + "_laccel_z_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    z_gyro_FFT_sum = np.load(file_path + "_gyro_z_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    norm_mag_FFT_sum = np.load(file_path + "_mag_norm_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    z_lacc_FFT = np.load(file_path + "_laccel_z_amplitude_frequency_range5Hz.npy")
    z_gyro_FFT = np.load(file_path + "_gyro_z_amplitude_frequency_range5Hz.npy")
    norm_mag_FFT = np.load(file_path + "_mag_norm_amplitude_frequency_range5Hz.npy")
    result = np.concatenate([xy_mean.reshape([-1, 1]), xy_var.reshape([-1, 1]), z_mean.reshape([-1, 1]), z_var.reshape([-1, 1]), z_skew.reshape([-1, 1]), z_kurtosis.reshape([-1, 1]), \
                             z_lacc_FFT_sum, z_gyro_FFT_sum, norm_mag_FFT_sum, z_lacc_FFT, z_gyro_FFT, norm_mag_FFT], axis=1)
    del xy_mean, xy_var, z_mean, z_var, z_skew, z_kurtosis, z_lacc_FFT_sum, z_gyro_FFT_sum, norm_mag_FFT_sum, z_gyro_FFT, norm_mag_FFT
    return result

In [ ]:
Bag = load_npy("Bag")
Hips = load_npy("Hips")
Torso = load_npy("Torso")
Hand = load_npy("Hand")

Hand.shape

In [ ]:
label = np.load("../Data/センサ別npyファイル/validation/validation_Bag/validation_Bag_Label.npy")[:, 0].reshape([-1, 1])

user2_Label = label[:14813]
user3_Label = label[14813:]

user2_Label.shape, user3_Label.shape

In [ ]:
user2_Bag = Bag[:14813]
user2_Hips = Hips[:14813]
user2_Torso = Torso[:14813]
user2_Hand = Hand[:14813]

user3_Bag = Bag[14813:]
user3_Hips = Hips[14813:]
user3_Torso = Torso[14813:]
user3_Hand = Hand[14813:]

In [ ]:
user2_Bag_train, user2_Bag_val = train_test_split(user2_Bag, test_size=0.3, shuffle=False)
user2_Hips_train, user2_Hips_val = train_test_split(user2_Hips, test_size=0.3, shuffle=False)
user2_Torso_train, user2_Torso_val = train_test_split(user2_Torso, test_size=0.3, shuffle=False)
user2_Hand_train, user2_Hand_val = train_test_split(user2_Hand, test_size=0.3, shuffle=False)

user3_Bag_train, user3_Bag_val = train_test_split(user3_Bag, test_size=0.3, shuffle=False)
user3_Hips_train, user3_Hips_val = train_test_split(user3_Hips, test_size=0.3, shuffle=False)
user3_Torso_train, user3_Torso_val = train_test_split(user3_Torso, test_size=0.3, shuffle=False)
user3_Hand_train, user3_Hand_val = train_test_split(user3_Hand, test_size=0.3, shuffle=False)

user2_Label_train, user2_Label_val = train_test_split(user2_Label, test_size=0.3, shuffle=False)
user3_Label_train, user3_Label_val = train_test_split(user3_Label, test_size=0.3, shuffle=False)

user2_Hand_train.shape, user2_Hand_val.shape, user3_Hand_train.shape, user3_Hand_val.shape, user2_Label_train.shape, user2_Label_val.shape

In [ ]:
def load_npy():
    file_path = "../Output/test/test"
    xy_mean = np.load(file_path + "_glo_laccel_xy_mean.npy")
    xy_var = np.load(file_path + "_glo_laccel_xy_var.npy")
    z_mean = np.load(file_path + "_glo_laccel_z_mean.npy")
    z_var = np.load(file_path + "_glo_laccel_z_var.npy")
    z_skew = np.load(file_path + "_glo_laccel_z_skew.npy")
    z_kurtosis = np.load(file_path + "_glo_laccel_z_kurtosis.npy")
    z_lacc_FFT = np.load(file_path + "_glo_laccel_z_amplitude_frequency_range5Hz.npy")[:, 0:-1:2]
    z_gyro_FFT = np.load(file_path + "_glo_gyro_z_amplitude_frequency_range5Hz.npy")[:, 0:-1:2]
    result = np.concatenate([xy_mean.reshape([-1, 1]), xy_var.reshape([-1, 1]), z_mean.reshape([-1, 1]), z_var.reshape([-1, 1]), z_skew.reshape([-1, 1]), z_kurtosis.reshape([-1, 1]), z_lacc_FFT, z_gyro_FFT], axis=1)
    del xy_mean, xy_var, z_mean, z_var, z_skew, z_kurtosis, z_gyro_FFT
    return result

In [ ]:
def load_npy():
    file_path = "../Output/test/test"    
    xy_mean = np.load(file_path + "_laccel_xy_mean.npy")
    xy_var = np.load(file_path + "_laccel_xy_var.npy")
    z_mean = np.load(file_path + "_laccel_z_mean.npy")
    z_var = np.load(file_path + "_laccel_z_var.npy")
    z_skew = np.load(file_path + "_laccel_z_skew.npy")
    z_kurtosis = np.load(file_path + "_laccel_z_kurtosis.npy")
    z_lacc_FFT_sum = np.load(file_path + "_laccel_z_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    z_gyro_FFT_sum = np.load(file_path + "_gyro_z_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    norm_mag_FFT_sum = np.load(file_path + "_mag_norm_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    z_lacc_FFT = np.load(file_path + "_laccel_z_amplitude_frequency_range5Hz.npy")
    z_gyro_FFT = np.load(file_path + "_gyro_z_amplitude_frequency_range5Hz.npy")
    norm_mag_FFT = np.load(file_path + "_mag_norm_amplitude_frequency_range5Hz.npy")
    result = np.concatenate([xy_mean.reshape([-1, 1]), xy_var.reshape([-1, 1]), z_mean.reshape([-1, 1]), z_var.reshape([-1, 1]), z_skew.reshape([-1, 1]), z_kurtosis.reshape([-1, 1]), \
                             z_lacc_FFT_sum, z_gyro_FFT_sum, norm_mag_FFT_sum, z_lacc_FFT, z_gyro_FFT, norm_mag_FFT], axis=1)
    del xy_mean, xy_var, z_mean, z_var, z_skew, z_kurtosis, z_lacc_FFT_sum, z_gyro_FFT_sum, norm_mag_FFT_sum, z_gyro_FFT, norm_mag_FFT
    return result

In [ ]:
test = load_npy()
test.shape

In [ ]:
Y_train = [0] * user2_Bag_train.shape[0] + [1] * user2_Bag_train.shape[0] + [2] * user2_Bag_train.shape[0] + [3] * user2_Bag_train.shape[0] + \
        [0] * user3_Bag_train.shape[0] + [1] * user3_Bag_train.shape[0] + [2] * user3_Bag_train.shape[0] + [3] * user3_Bag_train.shape[0]
Y_train = np.array(Y_train).reshape([-1, 1])

Y_val = [0] * user2_Bag_val.shape[0] + [1] * user2_Bag_val.shape[0] + [2] * user2_Bag_val.shape[0] + [3] * user2_Bag_val.shape[0] + \
        [0] * user3_Bag_val.shape[0] + [1] * user3_Bag_val.shape[0] + [2] * user3_Bag_val.shape[0] + [3] * user3_Bag_val.shape[0]
Y_val = np.array(Y_val).reshape([-1, 1])

Y_train.shape, Y_val.shape

In [ ]:
user2_Label_train = np.concatenate([user2_Label_train, user2_Label_train, user2_Label_train, user2_Label_train], axis=0)
user2_Label_val = np.concatenate([user2_Label_val, user2_Label_val, user2_Label_val, user2_Label_val], axis=0)
user3_Label_train = np.concatenate([user3_Label_train, user3_Label_train, user3_Label_train, user3_Label_train], axis=0)
user3_Label_val = np.concatenate([user3_Label_val, user3_Label_val, user3_Label_val, user3_Label_val], axis=0)

Y_train = np.concatenate([Y_train, np.concatenate([user2_Label_train, user3_Label_train], axis=0)], axis=1)
Y_val = np.concatenate([Y_val, np.concatenate([user2_Label_val, user3_Label_val], axis=0)], axis=1)

Y_train.shape, Y_val.shape

In [ ]:
X_train = np.concatenate([user2_Bag_train, user2_Hips_train, user2_Torso_train, user2_Hand_train, user3_Bag_train, user3_Hips_train, user3_Torso_train, user3_Hand_train])
X_val = np.concatenate([user2_Bag_val, user2_Hips_val, user2_Torso_val, user2_Hand_val, user3_Bag_val, user3_Hips_val, user3_Torso_val, user3_Hand_val])

X_train.shape, X_val.shape

In [ ]:
model = xgb.XGBClassifier(max_depth=18, min_child_weight=7, learning_rate=0.1, gamma=0.005, sub_sample=0.9, colsample_bytree=0.8, 
                          n_jobs=-1, tree_method='gpu_hist', gpu_id=0)
model.fit(X_train[(Y_train[:, 1] <= 3) & (Y_train[:, 1] > 1)], Y_train[(Y_train[:, 1] <= 3) & (Y_train[:, 1] > 1)][:, 0])

In [ ]:
def plot_confusion_matrix(test_y,pred_y,class_names,normalize=False):
    cm = confusion_matrix(test_y,pred_y)
    # classes = class_names[unique_labels(test_y,pred_y)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    fig, ax = plt.subplots(figsize=(16, 8))
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=class_names,
           yticklabels=class_names,
           ylabel='True label\n',
           xlabel='\nPredicted label')
    fmt = '.2f' if normalize else 'd'
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j,
                    i,
                    format(cm[i, j], fmt),
                    ha="center",
                    va="center",
                    color="red", fontsize=16)
    fig.tight_layout()
    return ax

In [ ]:
class_names = ['Bag', 'Hips', 'Torso', 'Hand']

predict = model.predict(X_val)
plot_confusion_matrix(Y_val[(Y_val[:, 1] <= 3) & (Y_val[:, 1] > 1)][:, 0], predict[(Y_val[:, 1] <= 3) & (Y_val[:, 1] > 1)], class_names, True)
plt.grid(False)

In [ ]:
predict = model.predict(test)

for i in range(len(class_names)):
    print("{}：{}".format(class_names[i], np.sum(predict==i)))

# Validationデータをすべて学習させる

In [ ]:
X = np.concatenate([X_train, X_val], axis=0)
Y = np.concatenate([Y_train, Y_val], axis=0)
X.shape, Y.shape

In [ ]:
model = xgb.XGBClassifier(max_depth=18, min_child_weight=7, learning_rate=0.1, gamma=0.005, sub_sample=0.9, colsample_bytree=0.8, 
                          n_jobs=-1, tree_method='gpu_hist', gpu_id=0)
model.fit(X[(Y[:, 1] <= 3) & (Y[:, 1] > 1)], Y[(Y[:, 1] <= 3) & (Y[:, 1] > 1)][:, 0])

In [ ]:
class_names = ['Bag', 'Hips', 'Torso', 'Hand']

predict = model.predict(X[(Y[:, 1] <= 3) & (Y[:, 1] > 1)])
plot_confusion_matrix(Y[(Y[:, 1] <= 3) & (Y[:, 1] > 1)][:, 0], predict, class_names, True)
plt.grid(False)

In [ ]:
predict = model.predict(test)

for i in range(len(class_names)):
    print("{}：{}".format(class_names[i], np.sum(predict==i)))

## ここで満を持してtestデータの出力確率が75%以上のWalkingかRunだと思われるサンプルだけで保持位置を推定！

In [ ]:
a = np.load("test_walking_run_index.npy")
a.shape

In [ ]:
predict = model.predict_proba(test[a])

for i in range(len(class_names)):
    print("{}：{}".format(class_names[i], np.sum(predict[:, i]>=0.75)))

In [ ]:
_, ax = plt.subplots(figsize=(12, 24))
xgb.plot_importance(model,
                    ax=ax,
                    importance_type='gain',
                    show_values=False)
plt.grid(False)

In [ ]:
%reset

# Trainデータで保持位置決定精度を調査

In [ ]:
import xgboost as xgb
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

import seaborn as sns
sns.set(font='Yu Gothic')
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
def load_npy(hold_position):
    file_path = "../Output/train/train_" + hold_position + "/train_" + hold_position
    xy_mean = np.load(file_path + "_glo_laccel_xy_mean.npy")
    xy_var = np.load(file_path + "_glo_laccel_xy_var.npy")
    z_mean = np.load(file_path + "_glo_laccel_z_mean.npy")
    z_var = np.load(file_path + "_glo_laccel_z_var.npy")
    z_skew = np.load(file_path + "_glo_laccel_z_skew.npy")
    z_kurtosis = np.load(file_path + "_glo_laccel_z_kurtosis.npy")
    z_lacc_FFT_sum = np.load(file_path + "_glo_laccel_z_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    z_gyro_FFT_sum = np.load(file_path + "_glo_gyro_z_ver2_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    norm_mag_FFT_sum = np.load(file_path + "_glo_mag_norm_ver2_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    z_lacc_FFT = np.load(file_path + "_glo_laccel_z_amplitude_frequency_range5Hz.npy")
    z_gyro_FFT = np.load(file_path + "_glo_gyro_z_ver2_amplitude_frequency_range5Hz.npy")
    norm_mag_FFT = np.load(file_path + "_glo_mag_norm_ver2_amplitude_frequency_range5Hz.npy")
    result = np.concatenate([xy_mean.reshape([-1, 1]), xy_var.reshape([-1, 1]), z_mean.reshape([-1, 1]), z_var.reshape([-1, 1]), z_skew.reshape([-1, 1]), z_kurtosis.reshape([-1, 1]), \
                             z_lacc_FFT_sum, z_gyro_FFT_sum, norm_mag_FFT_sum, z_lacc_FFT, z_gyro_FFT, norm_mag_FFT], axis=1)
    del xy_mean, xy_var, z_mean, z_var, z_skew, z_kurtosis, z_lacc_FFT_sum, z_gyro_FFT_sum, norm_mag_FFT_sum, z_gyro_FFT, norm_mag_FFT
    return result

In [ ]:
Bag = load_npy("Bag")
Hips = load_npy("Hips")
Torso = load_npy("Torso")
Hand = load_npy("Hand")

Hand.shape

In [ ]:
Label = np.load("../Data/センサ別npyファイル/train/train_Bag/train_Bag_Label.npy")[:, 0].reshape([-1, 1])
Label.shape

In [ ]:
Bag_train, Bag_val = train_test_split(Bag, test_size=0.7, shuffle=False)
Hips_train, Hips_val = train_test_split(Hips, test_size=0.7, shuffle=False)
Torso_train, Torso_val = train_test_split(Torso, test_size=0.7, shuffle=False)
Hand_train, Hand_val = train_test_split(Hand, test_size=0.7, shuffle=False)
Label_train, Label_val = train_test_split(Label, test_size=0.7, shuffle=False)

Hand_train.shape, Hand_val.shape

In [ ]:
Y_train = [0] * 58647 + [1] * 58647 + [2] * 58647 + [3] * 58647
Y_train = np.array(Y_train).reshape([-1, 1])

Y_val = [0] * 136844 + [1] * 136844 + [2] * 136844 + [3] * 136844
Y_val = np.array(Y_val).reshape([-1, 1])

Y_train.shape, Y_val.shape

In [ ]:
X_train = np.concatenate([Bag_train, Hips_train, Torso_train, Hand_train])
X_val = np.concatenate([Bag_val, Hips_val, Torso_val, Hand_val])

X_train.shape, X_val.shape

In [ ]:
Label_train = np.concatenate([Label_train, Label_train, Label_train, Label_train], axis=0)
Label_val = np.concatenate([Label_val, Label_val, Label_val, Label_val], axis=0)

Label_train.shape, Label_val.shape

In [ ]:
Y_train = np.concatenate([Y_train, Label_train], axis=1)
Y_val = np.concatenate([Y_val, Label_val], axis=1)

Y_train.shape, Y_val.shape

In [ ]:
model = xgb.XGBClassifier(max_depth=18, min_child_weight=7, learning_rate=0.1, gamma=0.005, sub_sample=0.9, colsample_bytree=0.8, 
                          n_jobs=-1, tree_method='gpu_hist', gpu_id=0, n_estimator=50)
model.fit(X_train[(Y_train[:, 1] > 1) & (Y_train[:, 1] <= 3)], Y_train[(Y_train[:, 1] > 1) & (Y_train[:, 1] <= 3)][:, 0])

In [ ]:
def plot_confusion_matrix(test_y,pred_y,class_names,normalize=False):
    cm = confusion_matrix(test_y,pred_y)
    # classes = class_names[unique_labels(test_y,pred_y)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    fig, ax = plt.subplots(figsize=(16, 8))
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=class_names,
           yticklabels=class_names,
           ylabel='True label\n',
           xlabel='\nPredicted label')
    fmt = '.2f' if normalize else 'd'
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j,
                    i,
                    format(cm[i, j], fmt),
                    ha="center",
                    va="center",
                    color="red", fontsize=16)
    fig.tight_layout()
    return ax

In [ ]:
class_names = ['Bag', 'Hips', 'Torso', 'Hand']

predict = model.predict(X_val[(Y_val[:, 1] > 1) & (Y_val[:, 1] <= 3)])
plot_confusion_matrix(Y_val[(Y_val[:, 1] > 1) & (Y_val[:, 1] <= 3)][:, 0], predict, class_names, True)
plt.grid(False)

In [ ]:
%reset

# trainデータ全部学習→validationデータの保持位置推定の精度を確認

In [ ]:
import numpy as np
import os
import random
import scipy
from scipy import signal
from numpy.fft import fft
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.initializers import he_normal
from tensorflow.python.keras.utils.vis_utils import plot_model

from sklearn.model_selection import train_test_split

import pandas as pd
import pickle

import seaborn as sns
sns.set(font='Yu Gothic')
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
Label = np.load("../Data/センサ別npyファイル/train/train_Bag/train_Bag_Label.npy")[:, 0].reshape([-1])
Label.shape

In [ ]:
Bag = np.load("../Output/train/train_Bag/train_Bag_lacc_norm_spectram.npy").reshape([-1, 1, 128, 25])
Hips = np.delete(np.load("../Output/train/train_Hips/train_Hips_lacc_norm_spectram.npy"), 120845, 0).reshape([-1, 1, 128, 25])
Torso = np.load("../Output/train/train_Torso/train_Torso_lacc_norm_spectram.npy").reshape([-1, 1, 128, 25])
Hand = np.load("../Output/train/train_Hand/train_Hand_lacc_norm_spectram.npy").reshape([-1, 1, 128, 25])

Hand.shape

In [ ]:
X_train = np.concatenate([Bag, Hips, Torso, Hand], axis=0)
Y_train = [0] * 195491 + [1] * 195490 + [2] * 195491 + [3] * 195491
Y_train = np.array(Y_train).reshape([-1, 1])

Label = np.concatenate([Label, np.delete(Label, 120845, 0), Label, Label], axis=0)

Y_train = np.concatenate([Y_train, Label], axis=1)
del Label
X_train.shape, Y_train.shape

In [ ]:
def load_npy(hold_position):
    file_path = "../Output/validation/validation_" + hold_position + "/validation_" + hold_position
    xy_mean = np.load(file_path + "_glo_laccel_xy_mean.npy")
    xy_var = np.load(file_path + "_glo_laccel_xy_var.npy")
    z_mean = np.load(file_path + "_glo_laccel_z_mean.npy")
    z_var = np.load(file_path + "_glo_laccel_z_var.npy")
    z_skew = np.load(file_path + "_glo_laccel_z_skew.npy")
    z_kurtosis = np.load(file_path + "_glo_laccel_z_kurtosis.npy")
    z_lacc_FFT_sum = np.load(file_path + "_glo_laccel_z_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    z_gyro_FFT_sum = np.load(file_path + "_glo_gyro_z_ver2_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    norm_mag_FFT_sum = np.load(file_path + "_glo_mag_norm_ver2_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    z_lacc_FFT = np.load(file_path + "_glo_laccel_z_amplitude_frequency_range5Hz.npy")
    z_gyro_FFT = np.load(file_path + "_glo_gyro_z_ver2_amplitude_frequency_range5Hz.npy")
    norm_mag_FFT = np.load(file_path + "_glo_mag_norm_ver2_amplitude_frequency_range5Hz.npy")
    result = np.concatenate([xy_mean.reshape([-1, 1]), xy_var.reshape([-1, 1]), z_mean.reshape([-1, 1]), z_var.reshape([-1, 1]), z_skew.reshape([-1, 1]), z_kurtosis.reshape([-1, 1]), \
                             z_lacc_FFT_sum, z_gyro_FFT_sum, norm_mag_FFT_sum, z_lacc_FFT, z_gyro_FFT, norm_mag_FFT], axis=1)
    del xy_mean, xy_var, z_mean, z_var, z_skew, z_kurtosis, z_lacc_FFT_sum, z_gyro_FFT_sum, norm_mag_FFT_sum, z_gyro_FFT, norm_mag_FFT
    return result

In [ ]:
Bag = load_npy("Bag")
Hips = load_npy("Hips")
Torso = load_npy("Torso")
Hand = load_npy("Hand")

Hand.shape

In [ ]:
X_val = np.concatenate([Bag, Hips, Torso, Hand], axis=0)
Y_val = [0] * 28685 + [1] * 28685 + [2] * 28685 + [3] * 28685
Y_val = np.array(Y_val).reshape([-1, 1])
Label = np.load("../Data/センサ別npyファイル/validation/validation_Bag/validation_Bag_Label.npy")[:, 0].reshape([-1, 1])
Label = np.concatenate([Label, Label, Label, Label], axis=0)

Y_val = np.concatenate([Y_val, Label], axis=1)
del Label
X_val.shape, Y_val.shape

In [ ]:
std = StandardScaler()

X_train[(Y_train[:, 1] > 1) | (Y_train[:, 1] <= 3)][:, :39] = std.fit_transform(X_train[(Y_train[:, 1] > 1) | (Y_train[:, 1] <= 3)][:, :39])
X_train[(Y_train[:, 1] > 1) | (Y_train[:, 1] <= 3)][:, [i for i in range(40, X_train.shape[1], 2)]] = std.fit_transform(X_train[(Y_train[:, 1] > 1) | (Y_train[:, 1] <= 3)][:, [i for i in range(40, X_train.shape[1], 2)]])

X_val[(Y_val[:, 1] > 1) | (Y_val[:, 1] <= 3)][:, :39] = std.fit_transform(X_val[(Y_val[:, 1] > 1) | (Y_val[:, 1] <= 3)][:, :39])
X_val[(Y_val[:, 1] > 1) | (Y_val[:, 1] <= 3)][:, [i for i in range(40, X_val.shape[1], 2)]] = std.fit_transform(X_val[(Y_val[:, 1] > 1) | (Y_val[:, 1] <= 3)][:, [i for i in range(40, X_val.shape[1], 2)]])

X_train.shape, X_val.shape

In [ ]:
learns = []
tests = []

for i in range(4):
    tmp = np.where(Y_train[:, 0] == i)[0]
    learns.append(tmp[:tmp.shape[0]//10*7])
    tests.append(tmp[tmp.shape[0]//10*7:])
    
learns, tests, len(learns[0]), len(tests[0])

In [ ]:
x_train = X_train[learns[0]]
y_train = Y_train[learns[0]]
x_val = X_train[tests[0]]
y_val = Y_train[tests[0]]

for i in range(1, len(learns)):
    x_train = np.concatenate([x_train, X_train[learns[i]]])
    y_train = np.concatenate([y_train, Y_train[learns[i]]])
    
    x_val = np.concatenate([x_val, X_train[tests[i]]])
    y_val = np.concatenate([y_val, Y_train[tests[i]]])
    
x_train.shape, y_train.shape, x_val.shape, y_val.shape

In [ ]:
evals_result = {}

model = xgb.XGBClassifier(max_depth=18, min_child_weight=7, learning_rate=0.1, gamma=0.005, sub_sample=0.9, colsample_bytree=0.8, 
                          n_jobs=-1, tree_method='gpu_hist', gpu_id=0, n_estimator=10000)
model.fit(x_train[(y_train[:, 1] > 1) & (y_train[:, 1] <= 3)], y_train[(y_train[:, 1] > 1) & (y_train[:, 1] <= 3)][:, 0], 
          early_stopping_rounds=30, 
          eval_set=[(x_train[(y_train[:, 1] > 1) & (y_train[:, 1] <= 3)], y_train[(y_train[:, 1] > 1) & (y_train[:, 1] <= 3)][:, 0]), 
                    (x_val[(y_val[:, 1] > 1) & (y_val[:, 1] <= 3)], y_val[(y_val[:, 1] > 1) & (y_val[:, 1] <= 3)][:, 0])], eval_metric='merror', verbose=False, 
          callbacks=[xgb.callback.record_evaluation(evals_result)])

In [ ]:
# 学習過程の名前は 'validation_{n}' になる
plt.figure(figsize=(24, 4))
train_metric = evals_result['validation_0']['merror']
plt.plot(train_metric, label='train merror')
eval_metric = evals_result['validation_1']['merror']
plt.plot(eval_metric, label='eval merror')
plt.grid()
plt.legend()
plt.xlabel('rounds(epochs)')
plt.ylabel('merror')
plt.xticks(np.arange(0, 150+1, 25))
plt.show()

In [ ]:
def plot_confusion_matrix(test_y,pred_y,class_names,normalize=False):
    cm = confusion_matrix(test_y,pred_y)
    # classes = class_names[unique_labels(test_y,pred_y)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    fig, ax = plt.subplots(figsize=(16, 8))
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=class_names,
           yticklabels=class_names,
           ylabel='True label\n',
           xlabel='\nPredicted label')
    fmt = '.2f' if normalize else 'd'
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j,
                    i,
                    format(cm[i, j], fmt),
                    ha="center",
                    va="center",
                    color="red", fontsize=16)
    fig.tight_layout()
    return ax

In [ ]:
class_names = ['Bag', 'Hips', 'Torso', 'Hand']

predict = model.predict(X_val[(Y_val[:, 1] > 1) & (Y_val[:, 1] <= 3)])
plot_confusion_matrix(Y_val[(Y_val[:, 1] > 1) & (Y_val[:, 1] <= 3)][:, 0], predict, class_names, True)
plt.grid(False)

In [ ]:
def load_npy():
    file_path = "../Output/test/test"    
    xy_mean = np.load(file_path + "_laccel_xy_mean.npy")
    xy_var = np.load(file_path + "_laccel_xy_var.npy")
    z_mean = np.load(file_path + "_laccel_z_mean.npy")
    z_var = np.load(file_path + "_laccel_z_var.npy")
    z_skew = np.load(file_path + "_laccel_z_skew.npy")
    z_kurtosis = np.load(file_path + "_laccel_z_kurtosis.npy")
    z_lacc_FFT_sum = np.load(file_path + "_laccel_z_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    z_gyro_FFT_sum = np.load(file_path + "_gyro_z_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    norm_mag_FFT_sum = np.load(file_path + "_mag_norm_sum_frequency_range5Hz.npy")[:, 0:-1:2]
    z_lacc_FFT = np.load(file_path + "_laccel_z_amplitude_frequency_range5Hz.npy")
    z_gyro_FFT = np.load(file_path + "_gyro_z_amplitude_frequency_range5Hz.npy")
    norm_mag_FFT = np.load(file_path + "_mag_norm_amplitude_frequency_range5Hz.npy")
    result = np.concatenate([xy_mean.reshape([-1, 1]), xy_var.reshape([-1, 1]), z_mean.reshape([-1, 1]), z_var.reshape([-1, 1]), z_skew.reshape([-1, 1]), z_kurtosis.reshape([-1, 1]), \
                             z_lacc_FFT_sum, z_gyro_FFT_sum, norm_mag_FFT_sum, z_lacc_FFT, z_gyro_FFT, norm_mag_FFT], axis=1)
    del xy_mean, xy_var, z_mean, z_var, z_skew, z_kurtosis, z_lacc_FFT_sum, z_gyro_FFT_sum, norm_mag_FFT_sum, z_gyro_FFT, norm_mag_FFT
    return result

In [ ]:
test = load_npy()
test.shape

In [ ]:
a = np.load("test_walking_run_index.npy")
a.shape

In [ ]:
test = test[a]
test.shape

In [ ]:
test[:, :39] = std.fit_transform(test[:, :39])
test[:, [i for i in range(40, test.shape[1], 2)]] = std.fit_transform(test[:, [i for i in range(40, test.shape[1], 2)]])

test.shape

In [ ]:
predict = model.predict_proba(X_val[(Y_val[:, 1] > 1) & (Y_val[:, 1] <= 3)])

for i in range(len(class_names)):
    print("{}：{}".format(class_names[i], np.sum(predict[:, i]>=0.70)))

In [ ]:
predict = model.predict_proba(test)

for i in range(len(class_names)):
    print("{}：{}".format(class_names[i], np.sum(predict[:, i]>=0.75)))

## trainデータ全部学習させて、validationデータをvalidationにする

In [ ]:
evals_result = {}

model = xgb.XGBClassifier(max_depth=18, min_child_weight=7, learning_rate=0.1, gamma=0.005, sub_sample=0.9, colsample_bytree=0.8, 
                          n_jobs=-1, tree_method='gpu_hist', gpu_id=0, n_estimator=10000)
model.fit(X_train[(Y_train[:, 1] > 1) & (Y_train[:, 1] <= 3)], Y_train[(Y_train[:, 1] > 1) & (Y_train[:, 1] <= 3)][:, 0], 
          early_stopping_rounds=30, 
          eval_set=[(X_train[(Y_train[:, 1] > 1) & (Y_train[:, 1] <= 3)], Y_train[(Y_train[:, 1] > 1) & (Y_train[:, 1] <= 3)][:, 0]), 
                    (X_val[(Y_val[:, 1] > 1) & (Y_val[:, 1] <= 3)], Y_val[(Y_val[:, 1] > 1) & (Y_val[:, 1] <= 3)][:, 0])], eval_metric='merror', verbose=False, 
          callbacks=[xgb.callback.record_evaluation(evals_result)])

In [ ]:
# 学習過程の名前は 'validation_{n}' になる
plt.figure(figsize=(24, 4))
train_metric = evals_result['validation_0']['merror']
plt.plot(train_metric, label='train merror')
eval_metric = evals_result['validation_1']['merror']
plt.plot(eval_metric, label='eval merror')
plt.grid()
plt.legend()
plt.xlabel('rounds(epochs)')
plt.ylabel('merror')
plt.xticks(np.arange(0, 150+1, 25))
plt.show()

In [ ]:
class_names = ['Bag', 'Hips', 'Torso', 'Hand']

predict = model.predict(X_val[(Y_val[:, 1] > 1) & (Y_val[:, 1] <= 3)])
plot_confusion_matrix(Y_val[(Y_val[:, 1] > 1) & (Y_val[:, 1] <= 3)][:, 0], predict, class_names, True)
plt.grid(False)

In [ ]:
predict = model.predict_proba(X_val[(Y_val[:, 1] > 1) & (Y_val[:, 1] <= 3)])

for i in range(len(class_names)):
    print("{}：{}".format(class_names[i], np.sum(predict[:, i]>=0.70)))

In [ ]:
predict = model.predict_proba(test)

for i in range(len(class_names)):
    print("{}：{}".format(class_names[i], np.sum(predict[:, i]>=0.75)))